# Tiling

### Before the data can be used in the model, they have to tiled to 256x256


https://towardsdatascience.com/efficiently-splitting-an-image-into-tiles-in-python-using-numpy-d1bf0dd7b6f7

TODO
1) Tif lesen, georeferenzierten Punkt oben links für späteres Nutzen erstellen 
2) JPEG erstellen
3) tiling
4) Klassifizierung 
5) wieder zusammensetzten der Tiles 
6) Georeferenzierung wieder hinzufügen 

In [74]:
import os
import sys
from PIL import Image
from osgeo import gdal
import numpy as np
import rasterio
from osgeo import osr
import geopandas as gpd
from shapely.geometry import Point
from pyproj import CRS

In [39]:
savedir = "E:/W_Katrin/Kigali/Raster/tiles"
filename = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif"

## Create Point for later Georeferencing

In [107]:
# get coordinates from raster
src = gdal.Open(filename)

ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform() # ulx, uly = upper left x and y values

# get projection from raster
wkt = src.GetProjection()
src_crs = osr.SpatialReference(wkt).GetAttrValue('AUTHORITY',1)
print("SRC_CRS:", src_crs)

# create Point 
point = Point(ulx, uly)
print("Point:", point)

#create new GeoDataFrame to save the point in
newdata = gpd.GeoDataFrame()
newdata['geometry'] = None
newdata.loc[0, 'geometry'] = point

# Set the GeoDataFrame's coordinate system to the same as the src raster 
newdata.crs = CRS.from_epsg(src_crs)
print("GeoDataFrame CRS:", newdata.crs)

#save GeoDataFrame
outfp = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_ReferencePointUL.shp"
newdata.to_file(outfp)

SRC_CRS: 32736
Point: POINT (173723.36172185623 9789230.701895246)
GeoDataFrame CRS: epsg:32736


In [ ]:
import subprocess as sp
command = "E:\W_Katrin\miniconda3\pkgs\gdal-3.4.1-py310h2a306c7_3\Scripts\gdal_edit.py -unsetgt E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_copy.tif"
os.system(command)

In [ ]:
import subprocess as sp

#locate your gdal_edit script
gdaledit = r"E:\W_Katrin\miniconda3\pkgs\gdal-3.4.1-py310h2a306c7_3\Scripts\gdal_edit.py"

#input data 
srcdata = 'E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_copy.tif'

cmd = [gdaledit, '-unsetgt', srcdata]
sp.check_call(cmd, shell=True)

In [65]:
!gdal_edit -unsetgt E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_copy.tif

Der Befehl "gdal_edit" ist entweder falsch geschrieben oder
konnte nicht gefunden werden.


## Convert Tif to JPEG
#### using gdal translate
https://gis.stackexchange.com/questions/42584/how-to-call-gdal-translate-from-python-code

In [17]:
# out_ds = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg"

In [67]:
!gdal_translate -of JPEG -b 1 -b 2 -b 3 -scale E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset.tif E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.jpg

Input file size is 13162, 8267
0...10...20...30...40...50...60...70...80...90...100 - done.

Warning 6: JPEG driver doesn't support data type Float32. Only eight bit byte bands supported.



In [ ]:
dst = gdal.Open("E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.jpg")
dst.GetMetadata()
!gdalinfo E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB_4.jpg

# TILING

In [69]:
# define tiling function 
def tiling(filename, savedir, target_size): 
    img = Image.open(filename)
    width, height = img.size
    start_pos = start_x, start_y = (0, 0)
    cropped_image_size = w, h = target_size
    frame_num = 1
    for col_i in range(0, width, w):
        for row_i in range(0, height, h):
            crop = img.crop((col_i, row_i, col_i + w, row_i + h))
            save_to= os.path.join(savedir, "testing_{row}_{col}.jpg".format(row=row_i, col=col_i))
            crop.save(save_to)


In [72]:
# run tiling
import warnings
savedir = "E:/W_Katrin/Kigali/Raster/tiles"
filename = "E:/W_Katrin/Kigali/Raster/2015_Pleiades_Kigali_subset_RGB.jpg"

warnings.simplefilter('ignore', Image.DecompressionBombWarning)

tiling(filename, savedir, target_size = (256,256))

## TODO

develop code to enter many images and return tiles in a logical way (also think of mask pixels!!) 

Problem: we have images, but we need the mask to be tiled at the same location the exact same way!